In [1]:
import pandas as pd

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
DIR = '/content/drive/My Drive/Colab Notebooks/data'
datafile = 'soc_dump.sql'
filename = DIR+'/'+datafile

In [8]:
import os
os.chdir(DIR)

In [27]:
import email
from email import policy
file_path ='/content/drive/My Drive/Colab Notebooks/data/eml/email.eml'
with open(file_path, 'rb') as f:
    # Use policy.default to easily handle different content types
    msg = email.message_from_binary_file(f, policy=policy.default)

# Extracting data
data = {
    "Subject": msg['subject'],
    "From": msg['from'],
    "To": msg['to'],
    "Date": msg['date'],
    "Body": msg.get_body(preferencelist=('plain')).get_content() if msg.get_body() else ""
}

# Create DataFrame
df = pd.DataFrame([data])
df.head()

,Subject,From,To,Date,Body
0,"Our free, hands-on data science courses",Kaggle <noreply@kaggle.com>,ckeerthana913@gmail.com,"Mon, 12 Jan 2026 11:21:16 -0800","Kaggle\n\n\n\n\n\n\nHi hghgf88fhv,\n\nAre you ..."


In [28]:
email_list = []
file_p ='/content/drive/My Drive/Colab Notebooks/data/eml'
if not os.path.exists(file_p):
    print(f"Directory not found: {file_p}")
else:
    for filename in os.listdir(file_p):
        if filename.endswith(".eml"):
            file_path = os.path.join(file_p, filename)

            with open(file_path, 'rb') as f:
                # policy.default is crucial for easy header/body access
                msg = email.message_from_binary_file(f, policy=policy.default)

                # Logic to extract the body safely
                body_content = ""
                try:
                    body = msg.get_body(preferencelist=('plain', 'html'))
                    if body:
                        body_content = body.get_content()
                except Exception:
                    body_content = "[Could not parse body]"

                email_list.append({
                    "Filename": filename,
                    "Subject": msg['subject'],
                    "From": msg['from'],
                    "To": msg['to'],
                    "Date": msg['date'],
                    "Body": body_content
                })

    # Create the DataFrame
    df = pd.DataFrame(email_list)

# Check the results
print(f"Total files processed: {len(df)}")
df.head()

Total files processed: 2


,Filename,Subject,From,To,Date,Body
0,Introducing Community Benchmarks on Kaggle!.eml,Introducing Community Benchmarks on Kaggle!,Kaggle <noreply@kaggle.com>,ckeerthana913@gmail.com,"Wed, 14 Jan 2026 07:36:02 -0800","Kaggle\n\n\n\n\n\n\n\n\nHi hghgf88fhv,\n\nToda..."
1,email.eml,"Our free, hands-on data science courses",Kaggle <noreply@kaggle.com>,ckeerthana913@gmail.com,"Mon, 12 Jan 2026 11:21:16 -0800","Kaggle\n\n\n\n\n\n\nHi hghgf88fhv,\n\nAre you ..."


In [33]:
with open(filename, 'r') as file:
    print(file.read())


-- Synthetic SOC database dump for lab use
CREATE DATABASE IF NOT EXISTS soc;
USE soc;

CREATE TABLE IF NOT EXISTS auth_logs (
  log_id INT PRIMARY KEY AUTO_INCREMENT,
  timestamp DATETIME,
  username VARCHAR(50),
  ip VARCHAR(45),
  status VARCHAR(10),
  role VARCHAR(20),
  country VARCHAR(50)
);

INSERT INTO auth_logs (timestamp, username, ip, status, role, country) VALUES
('2026-01-05 09:00:10', 'walter', '10.0.0.104', 'SUCCESS', 'admin', 'Germany'),
('2026-01-05 09:00:31', 'victor', '10.0.0.128', 'SUCCESS', 'user', 'USA'),
('2026-01-05 09:01:00', 'charlie', '10.0.0.162', 'SUCCESS', 'user', 'USA'),
('2026-01-05 09:01:34', 'trent', '10.0.0.99', 'SUCCESS', 'user', 'UAE'),
('2026-01-05 09:02:03', 'victor', '10.0.0.222', 'FAILED', 'user', 'USA'),
('2026-01-05 09:02:33', 'oscar', '10.0.0.228', 'SUCCESS', 'user', 'UK'),
('2026-01-05 09:03:02', 'bob', '10.0.0.77', 'SUCCESS', 'user', 'UAE'),
('2026-01-05 09:03:31', 'victor', '10.0.0.197', 'SUCCESS', 'admin', 'UAE'),
('2026-01-05 09:04:10', 

In [34]:
df.shape

(2, 6)

In [35]:
import sqlite3

In [36]:
with open(datafile, "r", encoding="utf-8") as f:
    sql = f.read()

In [37]:
import re

In [39]:
sql = re.sub(r"(?im)^\s*--.*$", "", sql)                 # remove comment lines starting with --
sql = re.sub(r"(?im)^\s*CREATE DATABASE.*?;\s*$", "", sql)
sql = re.sub(r"(?im)^\s*USE\s+\w+\s*;\s*$", "", sql)

In [40]:
sql = re.sub(
    r"(?i)\blog_id\s+INT\s+PRIMARY\s+KEY\s+AUTO_INCREMENT\b",
    "log_id INTEGER PRIMARY KEY AUTOINCREMENT",
    sql
)


In [42]:

conn = sqlite3.connect(":memory:")
conn.executescript(sql)


df = pd.read_sql("SELECT * FROM auth_logs;", conn)

df

,log_id,timestamp,username,ip,status,role,country
0,1,2026-01-05 09:00:10,walter,10.0.0.104,SUCCESS,admin,Germany
1,2,2026-01-05 09:00:31,victor,10.0.0.128,SUCCESS,user,USA
2,3,2026-01-05 09:01:00,charlie,10.0.0.162,SUCCESS,user,USA
3,4,2026-01-05 09:01:34,trent,10.0.0.99,SUCCESS,user,UAE
4,5,2026-01-05 09:02:03,victor,10.0.0.222,FAILED,user,USA
...,...,...,...,...,...,...,...
115,116,2026-01-05 09:57:38,walter,10.0.0.208,FAILED,user,Australia
116,117,2026-01-05 09:58:08,trent,10.0.0.213,FAILED,user,Singapore
117,118,2026-01-05 09:58:34,bob,10.0.0.73,FAILED,user,Australia
118,119,2026-01-05 09:59:09,walter,10.0.0.158,SUCCESS,user,Australia


In [45]:
df.columns

Index(['log_id', 'timestamp', 'username', 'ip', 'status', 'role', 'country'], dtype='object')

In [46]:
df.dtypes

,0
log_id,int64
timestamp,object
username,object
ip,object
status,object
role,object
country,object


In [63]:
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df.dtypes

,0
log_id,int64
timestamp,datetime64[ns]
username,object
ip,object
status,object
role,object
country,object


In [48]:
with open(filename, 'r') as file:
    sql_query=file.read()

In [49]:

with open(filename, 'r') as file:
  sql_query = file.read()

print(df)
df.shape

     log_id           timestamp username          ip   status   role  \
0         1 2026-01-05 09:00:10   walter  10.0.0.104  SUCCESS  admin   
1         2 2026-01-05 09:00:31   victor  10.0.0.128  SUCCESS   user   
2         3 2026-01-05 09:01:00  charlie  10.0.0.162  SUCCESS   user   
3         4 2026-01-05 09:01:34    trent   10.0.0.99  SUCCESS   user   
4         5 2026-01-05 09:02:03   victor  10.0.0.222   FAILED   user   
..      ...                 ...      ...         ...      ...    ...   
115     116 2026-01-05 09:57:38   walter  10.0.0.208   FAILED   user   
116     117 2026-01-05 09:58:08    trent  10.0.0.213   FAILED   user   
117     118 2026-01-05 09:58:34      bob   10.0.0.73   FAILED   user   
118     119 2026-01-05 09:59:09   walter  10.0.0.158  SUCCESS   user   
119     120 2026-01-05 09:59:36  mallory   10.0.0.48  SUCCESS   user   

       country  
0      Germany  
1          USA  
2          USA  
3          UAE  
4          USA  
..         ...  
115  Australia  

(120, 7)